In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

In [29]:
df_control=pd.read_excel("/kaggle/input/ab-testing-data/ab_testing.xlsx", sheet_name = "Control Group")
df_control.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


In [30]:
df_test=pd.read_excel("/kaggle/input/ab-testing-data/ab_testing.xlsx", sheet_name = "Test Group")
df_test.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [18]:
def data_review (dataframe):
    print("**Shape**")
    print(dataframe.shape)
    print("**Types**")
    print(dataframe.dtypes)
    print("**Describe**")
    print(dataframe.describe().T)
    print("**NA**")
    print(dataframe.isnull().sum())
data_review(df_control)
data_review(df_test)

**Shape**
(40, 4)
**Types**
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
**Describe**
            count           mean           std           min           25%  \
Impression   40.0  101711.449068  20302.157862  45475.942965  85726.690349   
Click        40.0    5100.657373   1329.985498   2189.753157   4124.304129   
Purchase     40.0     550.894059    134.108201    267.028943    470.095533   
Earning      40.0    1908.568300    302.917783   1253.989525   1685.847205   

                     50%            75%            max  
Impression  99790.701078  115212.816543  147539.336329  
Click        5001.220602    5923.803596    7959.125069  
Purchase      531.206307     637.957088     801.795020  
Earning      1975.160522    2119.802784    2497.295218  
**NA**
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int64
**Shape**
(40, 4)
**Types**
Impression    float64
Click         float64
Purchase      float64
Ear

In [38]:
df=pd.concat([df_control,df_test], axis=1)
columns = list(df.columns)
new_columns = []
for i, col in enumerate(columns):
    if i < 4:
        new_columns.append(col + "-" + "control")
    else:
        new_columns.append(col + "-" + "test")

df.columns = new_columns
df.head()


,Impression-control,Click-control,Purchase-control,Earning-control,Impression-test,Click-test,Purchase-test,Earning-test
0,82529.459271,6090.077317,665.211255,2311.277143,120103.503796,3216.547958,702.160346,1939.611243
1,98050.451926,3382.861786,315.084895,1742.806855,134775.943363,3635.082422,834.054286,2929.405820
2,82696.023549,4167.965750,458.083738,1797.827447,107806.620788,3057.143560,422.934258,2526.244877
3,109914.400398,4910.882240,487.090773,1696.229178,116445.275526,4650.473911,429.033535,2281.428574
4,108457.762630,5987.655811,441.034050,1543.720179,145082.516838,5201.387724,749.860442,2781.697521


In [39]:
#H0:M1=M2  ---> control and test groups mean are equal.
#H1:M1=!M2  ---> control and test groups mean are not equal.
print(df["Purchase-control"].mean())
print(df["Purchase-test"].mean())

550.8940587702316
582.1060966484677


H0 : Normal distribution assumption is provide.
H1 : Normal distribution assumption is not provide.

In [41]:
#H0 : Normal distribution assumption is provide. 
#H1 : Normal distribution assumption is not provide.
test_stat,pvalue=shapiro(df["Purchase-control"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
test_stat,pvalue=shapiro(df["Purchase-test"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891
Test Stat = 0.9589, p-value = 0.1541


H0 cannot be rejected because p-value > 0.05 for both.

In [43]:
#H0 : Variance is homogeneous. 
#H1 : Variance is not homogeneous.
test_stat,pvalue=levene(df["Purchase-control"],df["Purchase-test"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


H0 cannot be rejected because p-value > 0.05.

In [44]:
test_stat, pvalue = ttest_ind(df["Purchase-control"],df["Purchase-test"],
                              equal_var=True)
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


H0 cannot be rejected. There is no statistical difference in purchasing averages between the 2 data groups.